In [1]:
# Libraries
import websocket
import json
import requests
import time
from BinanceStreamHandler import BinanceStreamHandler
import redis

In [2]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)
data = r.hgetall("agg_trade_event")
print(data)

{}


In [16]:
# Base stream
base_url = 'wss://fstream.binance.com'
# Combined streams
combined_streams = 'wss://fstream.binance.com/stream' # output {"stream":"<streamName>","data":<rawPayload>}
# Aggregate Trade Streams: The Aggregate Trade Streams push market trade information that is aggregated for fills with same price and taking side every 100 milliseconds. Only market trades will be aggregated, which means the insurance fund trades and ADL trades won't be aggregated.
agg_trade_str_name = '<symbol>@aggTrade'
agg_trade_keys_map = {
  "e": 'Event type',
  "E": 'Event time',
  "s": 'Symbol',
  "a": 'Aggregate trade ID',
  "p": 'Price',
  "q": 'Quantity',
  "f": 'First trade ID',
  "l": 'Last trade ID',
  "T": 'Trade time',
  "m": 'Is the buyer the market maker?',
}

In [17]:
# def on_message(ws, message):
#     raw = json.loads(message)
#     # Get actual trade data from combined stream
#     if "data" in raw:
#         data = raw["data"]
#         stream = raw["stream"]
#     else:
#         data = raw
#         stream = "unknown"
#     # Rename keys using the mapping
#     mapped_data = {agg_trade_keys_map.get(k, k): v for k, v in data.items()}
#     mapped_data["stream"] = stream  # Keep stream name if needed
#     print(mapped_data)
# def on_error(ws, error):
#     print("Error:", error)
# def on_close(ws, close_status_code, close_msg):
#     print("Closed")
# def on_open(ws):
#     # Subscribe to BTC/USDT perpetual trades
#     payload = {
#         "method": "SUBSCRIBE",
#         "params": ["btcusdt@aggTrade"],
#         "id": 1
#     }
#     data = json.dumps(payload)
#     ws.send(data)
# ws = websocket.WebSocketApp(
#     combined_streams,
#     on_message=on_message,
#     on_error=on_error,
#     on_close=on_close,
#     on_open=on_open
# )
# ws.run_forever()

In [18]:

stream_list = ["btcusdt@aggTrade", "ethusdt@aggTrade"]
handler = BinanceStreamHandler(streams=stream_list, key_map=agg_trade_keys_map)
combined_stream_url = "wss://fstream.binance.com/stream?streams=" + "/".join(stream_list)
ws = websocket.WebSocketApp(
    combined_stream_url,
    on_message=handler.on_message,
    on_error=handler.on_error,
    on_close=handler.on_close,
    on_open=handler.on_open
)
ws.run_forever()


{'Event type': 'aggTrade', 'Event time': 1748538684345, 'Aggregate trade ID': 2738873420, 'Symbol': 'BTCUSDT', 'Price': '106643.10', 'Quantity': '0.016', 'First trade ID': 6353073849, 'Last trade ID': 6353073849, 'Trade time': 1748538684205, 'Is the buyer the market maker?': True, 'stream': 'btcusdt@aggTrade'}
{'result': None, 'id': 1, 'stream': 'unknown'}
{'Event type': 'aggTrade', 'Event time': 1748538684496, 'Aggregate trade ID': 2738873421, 'Symbol': 'BTCUSDT', 'Price': '106643.20', 'Quantity': '0.005', 'First trade ID': 6353073850, 'Last trade ID': 6353073851, 'Trade time': 1748538684341, 'Is the buyer the market maker?': False, 'stream': 'btcusdt@aggTrade'}
{'Event type': 'aggTrade', 'Event time': 1748538684670, 'Aggregate trade ID': 2228873862, 'Symbol': 'ETHUSDT', 'Price': '2643.05', 'Quantity': '0.075', 'First trade ID': 5739663163, 'Last trade ID': 5739663163, 'Trade time': 1748538684618, 'Is the buyer the market maker?': True, 'stream': 'ethusdt@aggTrade'}
{'Event type': 'ag

True